In [23]:
# Data analysis and manipulation
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Loadind dataset
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

target = train_df['Transported']

complete_df = pd.concat([train_df.drop('Transported', axis=1), test_df], axis=0, ignore_index=True)

# EDA - Spaceship Dataset

### Variables Qualitatives:
* Nominal: Name, PassengerId, HomePlanet, CryoSleep, Cabin, Destination, VIP, Transported
* Ordinal: -
 
### Variables Quantitatives:
* Continuous: RoomService, FoodCourt, ShoppingMall, Spa, VRDeck
* Discrete: Age

In [24]:
complete_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines


In [25]:
complete_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   12970 non-null  object 
 1   HomePlanet    12682 non-null  object 
 2   CryoSleep     12660 non-null  object 
 3   Cabin         12671 non-null  object 
 4   Destination   12696 non-null  object 
 5   Age           12700 non-null  float64
 6   VIP           12674 non-null  object 
 7   RoomService   12707 non-null  float64
 8   FoodCourt     12681 non-null  float64
 9   ShoppingMall  12664 non-null  float64
 10  Spa           12686 non-null  float64
 11  VRDeck        12702 non-null  float64
 12  Name          12676 non-null  object 
dtypes: float64(6), object(7)
memory usage: 1.3+ MB


In [26]:
complete_df.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,12700.000000,12707.000000,12681.000000,12664.000000,12686.000000,12702.000000
mean,28.771969,222.897852,451.961675,174.906033,308.476904,306.789482
std,14.387261,647.596664,1584.370747,590.558690,1130.279641,1180.097223
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,49.000000,77.000000,29.000000,57.000000,42.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [27]:
print(f'Dataset Nulls:\n\n{complete_df.isnull().sum()}')

Dataset Nulls:

PassengerId       0
HomePlanet      288
CryoSleep       310
Cabin           299
Destination     274
Age             270
VIP             296
RoomService     263
FoodCourt       289
ShoppingMall    306
Spa             284
VRDeck          268
Name            294
dtype: int64


In [28]:
complete_df['HomePlanet'].value_counts(dropna=False)

HomePlanet
Earth     6865
Europa    3133
Mars      2684
NaN        288
Name: count, dtype: int64

In [29]:
# New features based on PassengerID

complete_df['GroupId'] = complete_df['PassengerId'].str.split('_').str[0].astype(int)

group_sizes = complete_df['GroupId'].value_counts()
complete_df['GroupSize'] = complete_df['GroupId'].map(group_sizes)

In [30]:
# New features based on Cabin - Primero rellenar con Unknown igual se puede llenar mas tarde
complete_df['Cabin'] = complete_df['Cabin'].fillna('Unknown/Unknown/Unknown')

complete_df['CabinDeck'] = complete_df['Cabin'].str.split('/').str[0]
complete_df['CabinNum'] = pd.to_numeric(complete_df['Cabin'].str.split('/').str[1], errors="coerce").fillna(-1).astype(int)
complete_df['CabinSide'] = complete_df['Cabin'].str.split('/').str[2]

In [31]:
# New feature with the total spend on services, controlling the NaN
expenses_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

total_exp = complete_df[expenses_cols].fillna(0).sum(axis=1)
all_nan_mask = complete_df[expenses_cols].isna().all(axis=1)
complete_df['TotalExpense'] = total_exp.mask(all_nan_mask, np.nan)

In [81]:
complete_df.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,GroupId,GroupSize,CabinNum,TotalExpense
count,12700.000000,12800.000000,12790.000000,12795.000000,12793.000000,12793.000000,12970.000000,12970.000000,12970.000000,12970.000000
mean,28.771969,221.278359,448.109930,173.115279,305.896819,304.607207,4635.337471,2.022976,589.663917,1433.221049
std,14.387261,645.517250,1578.151525,587.791279,1125.893121,1176.174871,2685.904299,1.577102,515.001595,2807.369708
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,-1.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2306.250000,1.000000,156.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4632.000000,1.000000,409.000000,716.000000
75%,38.000000,47.000000,72.000000,26.000000,54.000000,40.000000,6924.750000,2.000000,992.000000,1442.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000,9280.000000,8.000000,1894.000000,35987.000000


In [79]:
# Revisamos si todas las personas del mismo grupo vienen del mismo planeta
group_planet_counts = complete_df.groupby("GroupId")["HomePlanet"].nunique().value_counts().reset_index()

group_planet_counts.columns = ['NumPlanetasDistintos', 'NumGrupos']
group_planet_counts

# Podemos rellenar los planetas nan de las personas del mismo grupo

,NumPlanetasDistintos,NumGrupos
0,1,9124
1,0,156


In [80]:
# Revisamos si todas las personas del mismo grupo van hacia el mismo planeta
group_dest_counts = complete_df.groupby('GroupId')['Destination'].nunique().value_counts().reset_index()

group_dest_counts.columns = ['NumDestinosDistintos', 'NumGrupos']
group_dest_counts

# Hay personas del mismo grupo con destinos diferentes

,NumDestinosDistintos,NumGrupos
0,1,8056
1,2,997
2,0,154
3,3,73


In [52]:
# Revisamos si hay cabinas con personas de diferentes grupos
real_cabins = complete_df[complete_df['Cabin'] != 'Unknown/Unknown/Unknown']

cabin_group_counts = real_cabins.groupby('Cabin')['GroupId'].nunique().value_counts().reset_index()

cabin_group_counts.columns = ['NumGruposDistintos', 'NumCabinas']
cabin_group_counts

# Todas las personas en una cabina son del mismo grupo

,NumGruposDistintos,NumCabinas
0,1,9825


In [75]:
# Revisamos si hay grupos con mas de una cabina
group_cabin_counts = complete_df.groupby("GroupId")["Cabin"].nunique().value_counts().reset_index()

group_cabin_counts.columns = ["NumCabinasDistintas", "NumGrupos"]
group_cabin_counts

# Hay grupos con más de una cabina

,NumCabinasDistintas,NumGrupos
0,1,8490
1,2,741
2,3,46
3,4,3


In [ ]:
def classify_group_cabin(cryo):
    n_true = (cryo == True).sum()
    n_false = (cryo == False).sum()
    n_nan = cryo.isna().sum()
    
    if n_true > 0 and n_false == 0 and n_nan == 0:
        return "Todos duermen"
    elif n_true == 0 and n_false > 0 and n_nan == 0:
        return "Todos despiertos"
    elif n_true > 0 and n_false > 0 and n_nan == 0:
        return "Durmiendo y despiertos"
    elif n_true > 0 and n_false == 0 and n_nan > 0:
        return "Todos duermen + NaN"
    elif n_true == 0 and n_false > 0 and n_nan > 0:
        return "Todos despiertos + NaN"
    elif n_true > 0 and n_false > 0 and n_nan > 0:
        return "Durmiendo y despiertos + NaN"
    else:
        return "Solo NaN"

In [68]:
# Revisamos si hay grupos con personas durmiendo y despiertas
group_class_nan = complete_df.groupby("GroupId")["CryoSleep"].apply(classify_group_cabin).value_counts().reset_index()

group_class_nan.columns = ['Tipos', 'NumGrupos']
group_class_nan.sort_values(by="NumGrupos", ascending=False).reset_index(drop=True)

,Tipos,NumGrupos
0,Todos despiertos,5454
1,Todos duermen,2480
2,Durmiendo y despiertos,1210
3,Solo NaN,52
4,Durmiendo y despiertos + NaN,40
5,Todos despiertos + NaN,28
6,Todos duermen + NaN,16


In [67]:
# Revisamos si hay cabinas con personas durmiendo y despiertas
real_cabins = complete_df[complete_df["Cabin"] != "Unknown/Unknown/Unknown"]
cabin_class_nan = real_cabins.groupby("Cabin")["CryoSleep"].apply(classify_group_cabin).value_counts().reset_index()

cabin_class_nan.columns = ['Tipos', 'NumCabinas']
cabin_class_nan.sort_values(by="NumCabinas", ascending=False).reset_index(drop=True)

,Tipos,NumCabinas
0,Todos despiertos,6067
1,Todos duermen,2710
2,Durmiendo y despiertos,915
3,Solo NaN,57
4,Durmiendo y despiertos + NaN,33
5,Todos despiertos + NaN,22
6,Todos duermen + NaN,21


In [82]:
# Aunque tanto para los grupos como para las cabinas la mayoria de sus integrantes toman la misma decisión en cuanto al CryoSleep
# Hay grupos y cabinas con personas durmiendo y personas despiertas mezcladas

In [35]:
# If you have expenses is because you are not in cryosleep
expense_nancryo_mask = (complete_df['TotalExpense'] > 0) & complete_df['CryoSleep'].isna()
complete_df.loc[expense_nancryo_mask, 'CryoSleep'] = False

In [41]:
# If you are in cryosleep you can't have expenses
is_cryo_mask = complete_df["CryoSleep"] == True
complete_df.loc[is_cryo_mask, expenses_cols] = complete_df.loc[is_cryo_mask, expenses_cols].fillna(0)
complete_df.loc[is_cryo_mask, 'TotalExpense'] = 0

In [42]:
print(f'Dataset Nulls:\n\n{complete_df.isnull().sum()}')

Dataset Nulls:

PassengerId       0
HomePlanet      288
CryoSleep       136
Cabin             0
Destination     274
Age             270
VIP             296
RoomService     170
FoodCourt       180
ShoppingMall    175
Spa             177
VRDeck          177
Name            294
GroupId           0
GroupSize         0
CabinDeck         0
CabinNum          0
CabinSide         0
TotalExpense      0
dtype: int64


In [43]:
# Encontrar familias por apellido y/o grupoID, seguro que llegan desde el mismo planeta y van hacia el mismo, podria completar cabina tambien
# Mismo grupo y mismo apellido -  Puede que misma cabina, mismo pais de destino e incluso cryo a la vez